<a href="https://colab.research.google.com/github/tselane2110/ML-Specialization/blob/main/StickerSalesForecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


Kaggle credentials set.
Kaggle credentials successfully validated.


In [3]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

playground_series_s5e1_path = kagglehub.competition_download('playground-series-s5e1')

print('Data source import complete.')


100%|██████████| 2.26M/2.26M [00:00<00:00, 108MB/s]

Extracting files...
Data source import complete.


In [7]:
print(playground_series_s5e1_path)

/root/.cache/kagglehub/competitions/playground-series-s5e1


## 1. Importing required libraries

In [4]:
import pandas as pd
import numpy as np

In [8]:
# getting the path to the files

import os
for dirname, _, filenames in os.walk('/root/.cache/kagglehub/competitions/playground-series-s5e1'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/root/.cache/kagglehub/competitions/playground-series-s5e1/test.csv
/root/.cache/kagglehub/competitions/playground-series-s5e1/sample_submission.csv
/root/.cache/kagglehub/competitions/playground-series-s5e1/train.csv


## 2. Importing Dataset

In [9]:
df = pd.read_csv("/root/.cache/kagglehub/competitions/playground-series-s5e1/train.csv")
df.head()

,id,date,country,store,product,num_sold
0,0,2010-01-01,Canada,Discount Stickers,Holographic Goose,NaN
1,1,2010-01-01,Canada,Discount Stickers,Kaggle,973.0
2,2,2010-01-01,Canada,Discount Stickers,Kaggle Tiers,906.0
3,3,2010-01-01,Canada,Discount Stickers,Kerneler,423.0
4,4,2010-01-01,Canada,Discount Stickers,Kerneler Dark Mode,491.0


## 3. Performing Exploratory Data Analysis

In [10]:
df.describe()

,id,num_sold
count,230130.000000,221259.000000
mean,115064.500000,752.527382
std,66432.953062,690.165445
min,0.000000,5.000000
25%,57532.250000,219.000000
50%,115064.500000,605.000000
75%,172596.750000,1114.000000
max,230129.000000,5939.000000


In [11]:
df.columns

Index(['id', 'date', 'country', 'store', 'product', 'num_sold'], dtype='object')

In [12]:
# checking for missing values
print(df.isna())

           id   date  country  store  product  num_sold
0       False  False    False  False    False      True
1       False  False    False  False    False     False
2       False  False    False  False    False     False
3       False  False    False  False    False     False
4       False  False    False  False    False     False
...       ...    ...      ...    ...      ...       ...
230125  False  False    False  False    False     False
230126  False  False    False  False    False     False
230127  False  False    False  False    False     False
230128  False  False    False  False    False     False
230129  False  False    False  False    False     False

[230130 rows x 6 columns]


In [13]:
df["num_sold"][0]

nan

In [14]:
rows_with_nan = df[df.isna().any(axis=1)]
rows_with_nan

,id,date,country,store,product,num_sold
0,0,2010-01-01,Canada,Discount Stickers,Holographic Goose,NaN
45,45,2010-01-01,Kenya,Discount Stickers,Holographic Goose,NaN
90,90,2010-01-02,Canada,Discount Stickers,Holographic Goose,NaN
135,135,2010-01-02,Kenya,Discount Stickers,Holographic Goose,NaN
180,180,2010-01-03,Canada,Discount Stickers,Holographic Goose,NaN
...,...,...,...,...,...,...
229905,229905,2016-12-29,Kenya,Discount Stickers,Holographic Goose,NaN
229950,229950,2016-12-30,Canada,Discount Stickers,Holographic Goose,NaN
229995,229995,2016-12-30,Kenya,Discount Stickers,Holographic Goose,NaN
230040,230040,2016-12-31,Canada,Discount Stickers,Holographic Goose,NaN


In [15]:
# checking for unique values for countries
countries = list(df.country.unique())
countries

['Canada', 'Finland', 'Italy', 'Kenya', 'Norway', 'Singapore']

In [16]:
# unique values for the type of stores
df.store.unique()

array(['Discount Stickers', 'Stickers for Less', 'Premium Sticker Mart'],
      dtype=object)

In [17]:
# unique values for the sticker/product sold
df['product'].unique()

array(['Holographic Goose', 'Kaggle', 'Kaggle Tiers', 'Kerneler',
       'Kerneler Dark Mode'], dtype=object)

In [18]:
# checking if we have any record of the number of stickers sold
# for the discount stickers

discount_sticker = df[df['store'] == 'Discount Stickers']
discount_sticker

,id,date,country,store,product,num_sold
0,0,2010-01-01,Canada,Discount Stickers,Holographic Goose,NaN
1,1,2010-01-01,Canada,Discount Stickers,Kaggle,973.0
2,2,2010-01-01,Canada,Discount Stickers,Kaggle Tiers,906.0
3,3,2010-01-01,Canada,Discount Stickers,Kerneler,423.0
4,4,2010-01-01,Canada,Discount Stickers,Kerneler Dark Mode,491.0
...,...,...,...,...,...,...
230115,230115,2016-12-31,Singapore,Discount Stickers,Holographic Goose,200.0
230116,230116,2016-12-31,Singapore,Discount Stickers,Kaggle,1259.0
230117,230117,2016-12-31,Singapore,Discount Stickers,Kaggle Tiers,931.0
230118,230118,2016-12-31,Singapore,Discount Stickers,Kerneler,556.0


### - Analyzing the rows with NaN values

In [19]:
rows_with_nan

,id,date,country,store,product,num_sold
0,0,2010-01-01,Canada,Discount Stickers,Holographic Goose,NaN
45,45,2010-01-01,Kenya,Discount Stickers,Holographic Goose,NaN
90,90,2010-01-02,Canada,Discount Stickers,Holographic Goose,NaN
135,135,2010-01-02,Kenya,Discount Stickers,Holographic Goose,NaN
180,180,2010-01-03,Canada,Discount Stickers,Holographic Goose,NaN
...,...,...,...,...,...,...
229905,229905,2016-12-29,Kenya,Discount Stickers,Holographic Goose,NaN
229950,229950,2016-12-30,Canada,Discount Stickers,Holographic Goose,NaN
229995,229995,2016-12-30,Kenya,Discount Stickers,Holographic Goose,NaN
230040,230040,2016-12-31,Canada,Discount Stickers,Holographic Goose,NaN


25.94% of our data has NaN values for the ```num_sold``` column.

In [20]:
# counting the values for each country

count_countries_nan = {}
for i in countries:
    count_countries_nan[i] = rows_with_nan[rows_with_nan["country"] == i].count()

print(count_countries_nan)

{'Canada': id          4246
date        4246
country     4246
store       4246
product     4246
num_sold       0
dtype: int64, 'Finland': id          0
date        0
country     0
store       0
product     0
num_sold    0
dtype: int64, 'Italy': id          0
date        0
country     0
store       0
product     0
num_sold    0
dtype: int64, 'Kenya': id          4625
date        4625
country     4625
store       4625
product     4625
num_sold       0
dtype: int64, 'Norway': id          0
date        0
country     0
store       0
product     0
num_sold    0
dtype: int64, 'Singapore': id          0
date        0
country     0
store       0
product     0
num_sold    0
dtype: int64}


* Canada has 4246 missing values for ```num_sold```
* Kenya has 4625 missing values for ```num_sold```

### - 1. Analyzing Missing Values for Canada

* Create a new df containing `date` and `num_sold` column for Canada only.
* Plot a timeseries and analyze it.
* Fill the missing values using **"Imputation Technique"**.

In [21]:
# analyzing missing values for Canada

### - 2. Analyzing Missing Values for Kenya

* Create a new df containing `date` and `num_sold` column for Kenya only.
* Plot a timeseries and analyze it.
* Fill the missing values using **"Imputation Technique"**.

In [22]:
# analyzing missing values for Kenya

### Next Steps:

* Create a new df which contains all the cols from the main original dataset but the rows for the Kenya and Canada are replaced by the new ones with the imputated values.